In [1]:
from cbrain.imports import *
from cbrain.data_generator import *
from cbrain.cam_constants import *
from cbrain.losses import *
from cbrain.utils import limit_mem
from cbrain.layers import *
from cbrain.data_generator import DataGenerator
import tensorflow as tf
from tensorflow import math as tfm
#import tensorflow_probability as tfp
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin
#import cartopy.crs as ccrs
import matplotlib.ticker as mticker
#from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import pickle
import sklearn
from sklearn.linear_model import LinearRegression
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
from cbrain.imports import *
from cbrain.utils import *
from cbrain.normalization import *
import h5py
from sklearn.preprocessing import OneHotEncoder
from cbrain.climate_invariant import *
import yaml
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# Load coordinates (just pick any file from the climate model run)
coor = xr.open_dataset("/DFS-L/DATA/pritchard/ankitesg/data/CESM2_f19_v13_updated_NN_pelayout01_ens_07.cam.h1.2003-01-22-00000.nc",\
                    decode_times=False)
lat = coor.lat; lon = coor.lon; lev = coor.lev;
DATA_DIR = '/DFS-L/DATA/pritchard/ankitesg/datav4/'
hyam = coor.hyam
hybm = coor.hybm
scale_dict = load_pickle('/export/nfs0home/ankitesg/tom/CBRAIN-CAM/nn_config/scale_dicts/2020_10_16_scale_dict_RG.pkl')['scale_dict_RG']

In [3]:
in_vars_RH = ['RH','TBP','CLDLIQBP','CLDICEBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars_RH =['QBCTEND','TBCTEND','CLDLIQBCTEND', 'CLDICEBCTEND', 'NN2L_FLWDS', 'NN2L_PRECC', 
            'NN2L_PRECSC', 'NN2L_SOLL', 'NN2L_SOLLD', 'NN2L_SOLS', 'NN2L_SOLSD', 'NN2L_NETSW']

TRAINFILE_RH = 'RG_RH_M4K_NORM_train_shuffle.nc'
NORMFILE_RH = 'RG_RH_M4K_NORM_norm.nc'
VALIDFILE_RH = 'RG_RH_M4K_NORM_valid.nc'
BASE_DIR = '/DFS-L/DATA/pritchard/ankitesg/'

In [4]:
train_gen_RH = DataGenerator(
    data_fn = f"{BASE_DIR}datav4/{TRAINFILE_RH}",
    input_vars = in_vars_RH,
    output_vars = out_vars_RH,
    norm_fn = f"{BASE_DIR}datav4/{NORMFILE_RH}",
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True
)

In [5]:
class DataGeneratorClimInvRealGeo(DataGenerator):

    def __init__(self, data_fn, input_vars, output_vars,
             norm_fn=None, input_transform=None, output_transform=None,
             batch_size=1024, shuffle=True, xarray=False, var_cut_off=None, normalize_flag=True,
             rh_trans=True,t2tns_trans=True,
             lhflx_trans=True,
             scaling=True,interpolate=True,
             hyam=None,hybm=None,
             inp_subRH=None,inp_divRH=None,
             inp_subTNS=None,inp_divTNS=None,
             lev=None, interm_size=40,
             lower_lim=6,
             is_continous=True,Tnot=5,
                mode='train', exp=None):
        self.scaling = scaling
        self.interpolate = interpolate
        self.rh_trans = rh_trans
        self.t2tns_trans = t2tns_trans
        self.lhflx_trans = lhflx_trans
        self.inp_shape = 64
        self.exp = exp
        self.mode=mode
        super().__init__(data_fn, input_vars,output_vars,norm_fn,input_transform,output_transform,
                        batch_size,shuffle,xarray,var_cut_off,normalize_flag) ## call the base data generator
        self.inp_sub = self.input_transform.sub
        self.inp_div = self.input_transform.div
        if self.rh_trans:
            self.qv2rhLayer = QV2RHNumpyReal(self.inp_sub,self.inp_div,inp_subRH,inp_divRH,hyam,hybm)
            


    def __getitem__(self, index):
        # Compute start and end indices for batch
        start_idx = index * self.batch_size
        end_idx = start_idx + self.batch_size

        # Grab batch from data
        batch = self.data_ds['vars'][start_idx:end_idx]

        # Split into inputs and outputs
        X = batch[:, self.input_idxs]
        Y = batch[:, self.output_idxs]
        # Normalize
        X_norm = self.input_transform.transform(X)
        X_result = X_norm
        Y = self.output_transform.transform(Y)
        if self.rh_trans:
            X_result = self.qv2rhLayer.process(X_result)
    
        return X_result, Y
    

In [6]:
in_vars = ['QBP','TBP','CLDLIQBP','CLDICEBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['QBCTEND','TBCTEND','CLDLIQBCTEND', 'CLDICEBCTEND', 'NN2L_FLWDS', 'NN2L_PRECC', 
            'NN2L_PRECSC', 'NN2L_SOLL', 'NN2L_SOLLD', 'NN2L_SOLS', 'NN2L_SOLSD', 'NN2L_NETSW']

In [7]:
TRAINFILE = 'RG_SP_M4K_train_shuffle.nc'
NORMFILE = 'RG_SP_M4K_NORM_norm.nc'
VALIDFILE = 'RG_SP_M4K_valid.nc'

In [8]:
train_gen = DataGeneratorClimInvRealGeo(
    data_fn = f'{DATA_DIR}{TRAINFILE}',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = f'{DATA_DIR}{NORMFILE}',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    rh_trans = True,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False,
    inp_subRH=train_gen_RH.input_transform.sub,inp_divRH=train_gen_RH.input_transform.div
)


In [15]:
train_gen.qv2rhLayer.hyam

array([0.00354464, 0.00738881, 0.01396721, 0.02394462, 0.03723029,
       0.05311461, 0.07005915, 0.07791257, 0.07660701, 0.07507109,
       0.07326415, 0.07113839, 0.06863753, 0.06569541, 0.06223416,
       0.05816217, 0.05337168, 0.04773593, 0.04110576, 0.0333057 ,
       0.02496844, 0.01709591, 0.01021471, 0.00480318, 0.00126068,
       0.        ])

In [16]:
hyam

<xarray.DataArray 'hyam' (lev: 26)>
array([0.003545, 0.007389, 0.013967, 0.023945, 0.03723 , 0.053115, 0.070059,
       0.077913, 0.076607, 0.075071, 0.073264, 0.071138, 0.068638, 0.065695,
       0.062234, 0.058162, 0.053372, 0.047736, 0.041106, 0.033306, 0.024968,
       0.017096, 0.010215, 0.004803, 0.001261, 0.      ])
Coordinates:
  * lev      (lev) float64 3.545 7.389 13.97 23.94 ... 867.2 929.6 970.6 992.6
Attributes:
    long_name:  hybrid A coefficient at layer midpoints

In [ ]:
HDEBUG:p,q,T,RH=   43690.7816151542       3.750179785378952E-004
   261.828829576347        10.86347

In [10]:
Q = []
val = 3.750179785378952E-004
for i in range(26):
    Q.append(val)
Q = np.array([Q])
T = []
val = 261.828829576347
for i in range(26):
    T.append(val)
T = np.array([T])
P = []
P.append(43690.7816151542)
P = np.array(P)

In [14]:
P.shape

(1,)

In [17]:
ThermLibNumpy.RHNumpy(T,Q,P0,P,train_gen.qv2rhLayer.hyam,train_gen.qv2rhLayer.hybm)

array([[0.00088043, 0.00183525, 0.00346921, 0.00594742, 0.00924735,
        0.01319274, 0.01740146, 0.0201689 , 0.0216223 , 0.02333216,
        0.02534372, 0.02771022, 0.03049428, 0.03376958, 0.03762281,
        0.04215592, 0.0474889 , 0.05376286, 0.06114387, 0.06982724,
        0.07910863, 0.08787269, 0.09553314, 0.1015575 , 0.10550116,
        0.10771242]], dtype=float32)

In [18]:
train_gen.qv2rhLayer.hyam

array([0.00354464, 0.00738881, 0.01396721, 0.02394462, 0.03723029,
       0.05311461, 0.07005915, 0.07791257, 0.07660701, 0.07507109,
       0.07326415, 0.07113839, 0.06863753, 0.06569541, 0.06223416,
       0.05816217, 0.05337168, 0.04773593, 0.04110576, 0.0333057 ,
       0.02496844, 0.01709591, 0.01021471, 0.00480318, 0.00126068,
       0.        ])